In [4]:
import json
import random
import pandas as pd
from datasets import Dataset
import transformers 
import evaluate

In [5]:
training_data = pd.read_json("/work/MarkusLundsfrydJensen#1865/Training_data/training_data.json")


In [6]:
subset_TD = training_data.sample(100)

In [7]:
len(subset_TD)

100

In [8]:
df = subset_TD

df.head()

,paragraph,sentence_nr,text,speaker,party,preceding_sentence,succeeding_sent,current_speaker_in_government,parties_in_government,date,label
1442409,22028,34,Der har jo altid været diskussioner her i Fol...,Poul Nyrup Rasmussen,S,"Jeg synes ikke, det er smukt at se på, og jeg ...","Folketinget har været her, om, hvem regeringen...",True,"[S, RV]",1999-02-11,0
268120,23743,1,"Når vi også har givet den rabat til DSB, som h...",Erling Christensen,S,"Kort og godt handler det om liberaliseringen, ...","Og mit spørgsmål gik jo ganske enkelt på, at d...",True,"[S, RV]",1999-03-19,0
813164,1225,5,Et af de centrale punkter i Enhedslistens fors...,Jan Petersen,S,"Det er i højere grad en debat om, hvilke midl...",Ministeren har svaret på spørgsmålet om den st...,True,"[S, RV]",1997-10-30,0
1010866,19079,15,"Panelet er sammensat af detailhandelen, forbr...",Pia Larsen,V,Det bliver ifølge lovforslaget op til et panel...,Panelets rolle som stopklods for ophævelse af...,False,"[S, RV]",1998-12-15,0
1832898,21409,4,Redegørelsen fra undervisningsministeren og f...,Klaus Kjær,DF,"Det er derfor også vigtigt, at vi lytter til ...",Redegørelsen er således et udtryk for regering...,False,"[S, RV]",1999-02-04,0


In [1]:
print('hello')

hello


In [9]:
# Keep only the necessary columns
df = df[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

# Convert to HuggingFace Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)


from transformers import AutoTokenizer

model_name = "bert-base-multilingual-cased"  # works for Danish
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(
        example['preceding_sentence'],
        example['text'],
        example['succeeding_sent']
    )

#tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=False)







Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:

#initialize model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

from sklearn.model_selection import train_test_split


# Split tokenized dataset
splits = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = splits['train']
test_dataset = splits['test']


#define metrics / Two options have been listed below. The second i probably preferred as ittakes the amount of 0 to 1' into account
# First option
import evaluate
import numpy as np

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

#second option
import torch
from collections import Counter

labels = df['label'].tolist()
class_counts = Counter(labels)
total = sum(class_counts.values())

# Higher weight = more emphasis
weights = [total/class_counts[0], total/class_counts[1]]
class_weights = torch.tensor(weights, dtype=torch.float)

#define custom trainer that uses weigted loss
from transformers import Trainer
import torch.nn as nn

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Define weighted loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

# And implement the second option in training
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()



#set up trainer
from transformers import TrainingArguments, Trainer

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./blame_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#train

trainer.train()



